In [27]:
import copy
import numpy as np

class Individuo:
    def __init__(self, alelos, longitud_gen, cromosoma):
        self._alelos = alelos
        self._longitud_gen = longitud_gen
        self._cromosoma = cromosoma
        self._fitness = 0

In [28]:
class AG:
    def __init__(self, cantidad_individuos, alelos, tamano_gen, generaciones, p, problema):
        self._cantidad_individuos = cantidad_individuos
        self._alelos = alelos
        self._tamano_gen = tamano_gen
        self._generaciones = generaciones
        self._p = p
        self._problema = problema
        self._individuos = np.array([])

    def run(self):
        self.crearIndividuos()
        self._mejor_historico = self._individuos[0]
        generacion = 1
        while generacion <= self._generaciones:
            self.evaluaIndividuos()
            hijos = np.array([])
            while len(hijos) < len(self._individuos):
                padre1 = self.ruleta()
                padre2 = self.ruleta()
                while padre1 == padre2:
                    padre2 = self.ruleta()
                h1, h2 = self.cruza(self._individuos[padre1], self._individuos[padre2])
                hijos = np.append(hijos, [h1])
                hijos = np.append(hijos, [h2])
            self.mutacion(hijos)
            self._individuos = np.copy(hijos)
            self._individuos[np.random.randint(len(self._individuos))] = copy.deepcopy(self._mejor_historico)
            if generacion % 200 == 0:
                print("Generación: ", generacion, 'Mejor Histórico: ', self._mejor_historico._cromosoma, self._mejor_historico._fitness)
            generacion += 1

    def crearIndividuos(self):
        for i in range(self._cantidad_individuos):
            cromosoma = np.random.randint(2, size = self._alelos)
            individuo = Individuo(self._alelos, self._tamano_gen, cromosoma)
            self._individuos = np.append(self._individuos, [individuo])

    def evaluaIndividuos(self):
        for i in self._individuos:
            i._fitness = self._problema.f(i._cromosoma)
            if i._fitness > self._mejor_historico._fitness:
                self._mejor_historico = copy.deepcopy(i)

    def ruleta(self):
        f_sum = np.sum([i._fitness for i in self._individuos])
        if f_sum < 0:
            f_sum *= -1
        if f_sum == 0:
            return np.random.randint(len(self._individuos))
        else:
            r = np.random.randint(f_sum + 1)
            k = 0
            F = self._individuos[k]._fitness
            if F < 0:
                F *= -1
            while F < r and k < len(self._individuos):
                k += 1
                if self._individuos[k]._fitness < 0:
                    F += self._individuos[k]._fitness * -1
                else:
                    F += self._individuos[k]._fitness
            return k

    def cruza(self, i1, i2):
        h1 = copy.deepcopy(i1)
        h2 = copy.deepcopy(i2)

        s = self._alelos - 1
        punto_cruza = np.random.randint(s) + 1
        h1._cromosoma[punto_cruza:], h2._cromosoma[punto_cruza:] = h2._cromosoma[punto_cruza:], h1._cromosoma[punto_cruza:]
        return h1, h2

    def mutacion(self, hijos):
        for h in hijos:
            for bit in range(len(h._cromosoma)):
                if np.random.rand() < self._p:
                    h._cromosoma[bit] = int(not h._cromosoma[bit])

In [29]:
class Monedas:
    def __init__(self, monedas):
        self._monedas = monedas

    def f(self, cromosoma):
        valorTotal = 0 #Para retornar la cantidad total obtenida
        penalizacion = 0 #La cantidad que penalizaremos si se toman monedas adyacentes
        adyacenteTomada = False #Para si agarró una moneda con monedas adyacentes tomadas

        for i, gen in enumerate(cromosoma): #Recorre el arreglo de monedas tomadas
            if gen: #Si la moneda está tomada
              valorTotal += self._monedas[i] #Suma su vlaor
              penalizacion += self._monedas[i] if adyacenteTomada else 0 #Si hay una adyacente tomada, suma a la penalización, caso contrario, no aumenta.
              adyacenteTomada = True #Se define como tomada porque la siguiente es adyacnete.
            else:
              adyacenteTomada = False #No hay adyacencia. Se la moneda de la siguiente vuelta...
        return valorTotal - (penalizacion + (penalizacion * .10)) #Suma de las monedas tomadas - (cantidad acumulada de la penalizacion + 10% cantidad acumulada de la penalización)

In [ ]:
def main():
    monedas = [  1, 20,  5, 1,  2,  5, 5,  1,  5,  2,  2,  1, 10, 5, 10,  5, 20, 20, 20,  5,  1,  1, 20,
                20,  1, 10, 2, 10,  5, 2, 10,  1, 20,  1, 20, 10, 5,  5, 20,  2, 10,  1,  2,  5, 10, 20,
                10,  2,  5, 5, 20,  1, 1,  5, 10, 10, 10,  1,  5, 2,  1,  2, 10, 20,  2, 10, 10, 20,  5,
                10,  1,  2, 1,  5, 20, 2,  5,  1, 5, 10, 2, 5, 10, 2, 1, 1, 1, 10, 20, 10, 20, 2, 2, 10,
                20, 10, 1, 1, 5, 2 ]
    filaMonedas = Monedas(monedas)
    alelos = len(monedas)
    individuos = 32
    tamano_gen = 1 #cada bit representa una moneda tomada
    generaciones = 2000
    factor_mutacion = 0.01
    ag = AG(individuos, alelos, tamano_gen, generaciones, factor_mutacion, filaMonedas)
    ag.run()
main()

In [ ]:
#----Para probar si los resultados tienen sentido----
arr = [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,]
input = [  1, 20,  5, 1,  2,  5, 5,  1,  5,  2,  2,  1, 10, 5, 10,  5, 20, 20, 20,  5,  1,  1, 20,
                20,  1, 10, 2, 10,  5, 2, 10,  1, 20,  1, 20, 10, 5,  5, 20,  2, 10,  1,  2,  5, 10, 20,
                10,  2,  5, 5, 20,  1, 1,  5, 10, 10, 10,  1,  5, 2,  1,  2, 10, 20,  2, 10, 10, 20,  5,
                10,  1,  2, 1,  5, 20, 2,  5,  1, 5, 10, 2, 5, 10, 2, 1, 1, 1, 10, 20, 10, 20, 2, 2, 10,
                20, 10, 1, 1, 5, 2 ]
arregloPosiciones = []
arregloValores = []
total = 0
for i, tomada in enumerate(arr):
  if(tomada == 1):
    arregloPosiciones.append(i)
    arregloValores.append(input[i])
    total += input[i]
print(f'Tamaño del arreglo de salida: {len(arr)}\nTamaño del arreglo de entrada: {len(input)}')
print(f'Tamaño del arreglo de posiciones: {len(arregloPosiciones)}\nTamaño del arreglo de valores: {len(arregloValores)}')
print(f'Posiciones: {arregloPosiciones}\nValores: {arregloValores}\nTotal: {total}')
